In [1]:
path = '/Users/longbro/Downloads/小贞毕设/'

In [2]:
# 读取csv
cost = spark.read.format('csv').load(path + 'cost.csv',header = True)
trans = spark.read.csv(path + 'trans.csv',header = True)
product = spark.read.format('csv').load(path + 'product.csv',header = True)
customer = spark.read.csv(path + 'customer.csv',header = True)
# 列名重命名1
oldColumns  = trans.columns
newColumns = ['account','trans_id','trans_name','trans_date','prod_id','prod_name','trans_way','amount']
trans = reduce(lambda trans, idx : trans.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)),trans)
# 列名重命名2
from pyspark.sql.functions import *
cost = cost.select(col("客户号").alias("account"),col("产品代码").alias("prod_id"),col("投资成本").alias("amount"))

# oldColumns2 = customer.columns
# newColumns2 = ['account','cus_type','cus_group','certification','gender','risk','birthday','manager']
# customer = reduce(lambda customer,idx: customer.withColumnRenamed(oldColumns2[idx],newColumns2[idx]),\
#                  xrange(len(oldColumns2)),customer)
# 列重命名 3
customer = customer.toDF('account','cus_type','cus_group','certification','gender','risk','birthday','manager')

oldColumns3 = product.columns
newColumns3 = [x.strip() for x in product.columns]
product = reduce(lambda p, idx:p.withColumnRenamed(oldColumns3[idx], newColumns3[idx]),\
                xrange(len(oldColumns3)),product)

# 注册为table1
from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)
sqlCtx.registerDataFrameAsTable(cost,'cost')

# spark.registerTempTable(trans,'trans')
# sqlCtx.dropTempTable('trans')

# 注册为table2
trans.registerTempTable('trans')
product.registerTempTable('product')
customer.registerTempTable('customer')
# spark.registerDataFrameAsTable

In [99]:
import pandas as pd
cost = pd.read_csv(path + 'cost.csv')
cost.columns = ['account','prod_id','amount']
# pandas 和 pyspark的 df 转换
cost = spark.createDataFrame(cost)
cost.toPandas().head(3)

,account,prod_id,amount
0,1000000132,942,4500
1,1000000132,1180,4500
2,1000000132,823,47000


In [112]:
# 创建一个新的session， 内存和table是共享的，其余是各自的
spark2 = spark.newSession()
# spark2.stop()

In [11]:
# range
spark.range(1,6,2).toPandas()

# sql
spark.sql("select * from cost limit 5").toPandas()

# 返回一个table为df
spark.table('cost').toPandas().head(3)

# udf
from pyspark.sql.types import *
def amount_div_10(x):
    return x / 10
sqlCtx.registerFunction('myudf', amount_div_10,DoubleType())
spark.sql("select account, myudf(amount) from cost").toPandas()

IllegalArgumentException: u"Error while instantiating 'org.apache.spark.sql.hive.HiveSessionState':"

In [3]:
from pyspark.sql import types
import pyspark.sql.functions as func
# groupby + agg聚合函数
# alias 别名、重命名
# (func.col("trans_way") == '网上银行') & 
judge = (func.col("amount")>0) 
# filter  =  where
trans = trans.filter(judge)
account_amount = trans.groupBy('account').agg(\
                    sum('amount').alias("total_amount"), \
                    func.format_number(func.avg('amount'),0).alias("avg_amount"), \
                    func.count('prod_id').alias("buy_times"),\
                    func.countDistinct("prod_id").alias("buy_types"),\
                    func.countDistinct('trans_way').alias("trans_ways"))\
                    .orderBy(['total_amount', 'avg_amount'],ascending = [False,True])
print account_amount.count()
account_amount.toPandas().head(3)
# ?func.format_number()

9362


,account,total_amount,avg_amount,buy_times,buy_types,trans_ways
0,1000599845,15073137,"3,014,627",5,2,1
1,1001099733,6810000,"6,810,000",1,1,1
2,1002856447,6180000,"2,060,000",3,1,2


In [17]:
# view 操作
cost.createTempView('cost_try')
cost = spark.sql('select * from cost_try limit 5')
cost.createOrReplaceTempView("cost_try")
spark.catalog.dropTempView("cost_try")

In [19]:
##### from pyspark.sql.types import IntegerType
# 这种方式没法修改到schema
# customer.select(customer.account.astype(IntegerType()))
# 对列进行类型转换
customer = customer.withColumn('year',substring('birthday',0,4).cast('int')).withColumn('month',substring('birthday',5,2).cast('int'))\
        .withColumn('age',(2017 - col('year')).cast('double')).withColumnRenamed('year','years')
#             .withColumn('birthday',unix_timestamp(col('birthday'), 'yyyyMMdd'))\
            
customer.show(5)

+----------+--------+---------+-------------+------+----+--------+-------+-----------+-----+-----+----+
|   account|cus_type|cus_group|certification|gender|risk|birthday|manager|      birth|years|month| age|
+----------+--------+---------+-------------+------+----+--------+-------+-----------+-----+-----+----+
|1000162168|      个人|     普通客户|          身份证|     女| 保守型|19560919|6000493|1.9560929E7| 1956|    9|61.0|
|1000162203|      个人|     普通客户|          身份证|     男| 保守型|19820630|       | 1.982064E7| 1982|    6|35.0|
|1000190754|      个人|     普通客户|          身份证|     女| 保守型|19551211|       |1.9551221E7| 1955|   12|62.0|
|1000190759|      个人|     普通客户|          身份证|     男| 保守型|19550529|       |1.9550539E7| 1955|    5|62.0|
|1000190779|      个人|     普通客户|          身份证|     女| 保守型|19831222|       |1.9831232E7| 1983|   12|34.0|
+----------+--------+---------+-------------+------+----+--------+-------+-----------+-----+-----+----+
only showing top 5 rows



In [20]:
# 交叉表
customer.crosstab('cus_type','cus_group').show()
# 类似unique
customer.freqItems(['cus_type','cus_group']).show()
# 按列运算，复制到新列
customer = customer.withColumn('birth',customer.birthday + 10)
# 透视表
customer.groupBy('cus_group','cus_type').pivot("gender").mean("age").na.fill({'男':1}).toPandas()
# cube 多维度观察数据 - cube的结果很奇怪
# 和groupby后的结果是不同的
customer.cube('cus_group',col('cus_type')).count().toPandas()

customer.groupBy('cus_group','cus_type').count().toPandas()

+------------------+----+---+---+
|cus_type_cus_group|普通客户| 白金| 钻石|
+------------------+----+---+---+
|                个人|9932|289| 45|
|                机构|  37|  0|  0|
+------------------+----+---+---+

+------------------+-------------------+
|cus_type_freqItems|cus_group_freqItems|
+------------------+-------------------+
|          [个人, 机构]|     [钻石, 普通客户, 白金]|
+------------------+-------------------+



,cus_group,cus_type,女,男
0,普通客户,机构,-2034.837838,1.000000
1,钻石,个人,51.285714,55.294118
2,普通客户,个人,48.623281,44.321323
3,白金,个人,57.449275,49.536585


In [78]:
# describe 基本是只针对连续值
customer.describe(['age','cus_group']).toPandas()
# distinct前必须是df
customer.select('age').distinct().toPandas()
# drop
customer.drop(col('manager')).toPandas()
# fillna - 这里的貌似是''，没填补，用replace  - 这里是一个空格
customer.na.fill({'manager':'haha'}).toPandas().head()
customer.replace(' ','haha','manager').toPandas().head()

,account,cus_type,cus_group,certification,gender,risk,birthday,manager,birth,years,month,age
0,1000162168,个人,普通客户,身份证,女,保守型,19560919,6000493,19560929,1956,9,61
1,1000162203,个人,普通客户,身份证,男,保守型,19820630,haha,19820640,1982,6,35
2,1000190754,个人,普通客户,身份证,女,保守型,19551211,haha,19551221,1955,12,62
3,1000190759,个人,普通客户,身份证,男,保守型,19550529,haha,19550539,1955,5,62
4,1000190779,个人,普通客户,身份证,女,保守型,19831222,haha,19831232,1983,12,34


In [93]:
# 过滤条件
judge = ((func.col('account') == '1000599845') | (func.col('prod_id') == '000289'))
trans.filter(judge).show(2)

# first
trans.first()

# foreach
# trans.account.foreach(lambda x:print x)

# sample ， sampleBy(without replacement) - 按列按值抽样
print trans.count()
print trans.sample(False, fraction= 0.5,seed = 10).count()
trans.sampleBy(col = 'trans_date',fractions = {'20161107':0.1,'20161017':0.5}).count()

+----------+--------+----------+----------+-------+---------+---------+--------+
|   account|trans_id|trans_name|trans_date|prod_id|prod_name|trans_way|  amount|
+----------+--------+----------+----------+-------+---------+---------+--------+
|1000584205|  100200|    基金产品购买|  20161107| 000289| 鹏华丰泰定开债A|     网上银行|100000.0|
|1000869570|  100200|    基金产品购买|  20161107| 000289| 鹏华丰泰定开债A|     网上银行| 50000.0|
+----------+--------+----------+----------+-------+---------+---------+--------+
only showing top 2 rows

34909
17666


615

In [63]:
trans2 = trans
oldColumns  = trans.columns
newColumns = ['account','trans_id','trans_name','trans_date','prod_id','prod_name','trans_way','amount']
newColumns = [name + '_1' for name in newColumns]
trans2 = reduce(lambda trans, idx : trans.withColumnRenamed(oldColumns[idx], newColumns[idx]), xrange(len(oldColumns)),trans)
# join 看看
cond =(trans.account == trans2.account_1) & (trans.prod_id == trans2.prod_id_1)
print trans.join(trans2, cond , 'left').count()
print trans.count()
# trans.join(trans, cond , 'left').toPandas().head()

164097
DataFrame[account: string, prod_id: string, count: bigint]
34909


In [109]:
# Window类下面的方法加上functions下的方法，基本可以实现SQL中的很多需要的方法

from pyspark.sql import Window
window = Window.partitionBy('account').orderBy('amount')
# .rangeBetween(0,-1)
trans.select(row_number().over(window)).show(1)

+-----------------------------------------------------------------------------------------+
|row_number() OVER (PARTITION BY account ORDER BY amount ASC NULLS FIRST UnspecifiedFrame)|
+-----------------------------------------------------------------------------------------+
|                                                                                        1|
+-----------------------------------------------------------------------------------------+
only showing top 1 row



In [112]:
# 排序 - 多种， orderBy是全量排序，sort是partition
# persist()持久化后，不需要每次进行计算
trans_order_limit = trans.orderBy(['account', 'prod_id'],ascending = [1, 0]).limit(10).persist()
trans_order_limit.printSchema()
# mode ， header 。。。
trans_order_limit.write.csv(path + 'try.csv')
trans_order_limit.write.format('csv').save(path + 'try2.csv')
trans_order_limit.unpersist()
# 再读
aa = spark.read.csv(path + 'try.csv')
aa.show()

root
 |-- account: string (nullable = true)
 |-- trans_id: string (nullable = true)
 |-- trans_name: string (nullable = true)
 |-- trans_date: string (nullable = true)
 |-- prod_id: string (nullable = true)
 |-- prod_name: string (nullable = true)
 |-- trans_way: string (nullable = true)
 |-- amount: string (nullable = true)

+----------+------+------+--------+------+---------+----+-----+
|       _c0|   _c1|   _c2|     _c3|   _c4|      _c5| _c6|  _c7|
+----------+------+------+--------+------+---------+----+-----+
|1000000132|200208|定期定额处理|20161017|162212|   泰达红利先锋|手机银行|500.0|
|1000000132|200208|定期定额处理|20161117|162212|   泰达红利先锋|手机银行|500.0|
|1000000132|200208|定期定额处理|20161219|162212|   泰达红利先锋|手机银行|500.0|
|1000000132|200208|定期定额处理|20161107|001180|广发医药卫生联接A|网上银行|500.0|
|1000000132|200208|定期定额处理|20161207|001180|广发医药卫生联接A|网上银行|500.0|
|1000000132|200208|定期定额处理|20161024|001150|融通互联网传媒前端|网上银行|500.0|
|1000000132|200208|定期定额处理|20161107|001150|融通互联网传媒前端|网上银行|500.0|
|1000000132|200208|定期定额处理|201611

In [36]:
from pyspark.ml.feature import *
# 二值化 - inputCol必须转换为double类型 , outputCol必须没有过这一列
customer = customer.withColumn('age',col('age').cast('double'))
binarizer = Binarizer(threshold=30, inputCol="age",outputCol="binarized_age")
customer = binarizer.transform(customer)
customer.show(2)

+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+
|   account|cus_type|cus_group|certification|gender|risk|birthday|manager|years|month| age|binarized_age|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+
|1000162168|      个人|     普通客户|          身份证|     女| 保守型|19560919|6000493| 1956|    9|61.0|          1.0|
|1000162203|      个人|     普通客户|          身份证|     男| 保守型|19820630|       | 1982|    6|35.0|          1.0|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+
only showing top 2 rows



In [37]:
# 连续值分箱 - 从0开始  - inputCol必须转换为double类型 , outputCol必须没有过这一列
customer = customer.withColumn('age',col('age').cast('double'))
split_list = [-float("inf"), 0,20,30,40,60,float("inf")]
buckets = Bucketizer(splits = split_list, inputCol="age", outputCol="bucketed_age")
customer = buckets.transform(customer)
customer.show(2)

+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+
|   account|cus_type|cus_group|certification|gender|risk|birthday|manager|years|month| age|binarized_age|bucketed_age|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+
|1000162168|      个人|     普通客户|          身份证|     女| 保守型|19560919|6000493| 1956|    9|61.0|          1.0|         5.0|
|1000162203|      个人|     普通客户|          身份证|     男| 保守型|19820630|       | 1982|    6|35.0|          1.0|         3.0|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+
only showing top 2 rows



In [38]:
# StringIndexer - LabelEncoder 先fit 再transform
# 缺失值填充
customer = customer.na.fill({'risk':'haha'})
stringIndex = StringIndexer(inputCol='risk', outputCol='risk_index')
customer = stringIndex.fit(customer).transform(customer)

# customer.select(col('cus_type')).distinct().collect()

AttributeError: 'OneHotEncoder' object has no attribute 'fit'

In [43]:
# OneHotEncoder
customer = customer.drop(col('risk_Vec'))
ohEncoder = OneHotEncoder(inputCol='risk_index', outputCol='risk_Vec')
customer = ohEncoder.transform(customer)
customer.show(2)

+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+----------+-------------+
|   account|cus_type|cus_group|certification|gender|risk|birthday|manager|years|month| age|binarized_age|bucketed_age|risk_index|     risk_Vec|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+----------+-------------+
|1000162168|      个人|     普通客户|          身份证|     女| 保守型|19560919|6000493| 1956|    9|61.0|          1.0|         5.0|       0.0|(3,[0],[1.0])|
|1000162203|      个人|     普通客户|          身份证|     男| 保守型|19820630|       | 1982|    6|35.0|          1.0|         3.0|       0.0|(3,[0],[1.0])|
+----------+--------+---------+-------------+------+----+--------+-------+-----+-----+----+-------------+------------+----------+-------------+
only showing top 2 rows



In [53]:
# StandardScaler  -  貌似输入列得是个matrix
trans = trans.na.fill({'amount':0}).withColumn('amount',col('amount').cast('double'))
scaler = StandardScaler(inputCol = 'amount', outputCol='amountScaler', withMean=False, withStd=True)
scalerModel =  scaler.fit(trans)
trans = scalerModel.transform(trans)
trans.show(5)

IllegalArgumentException: u'requirement failed: Column amount must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually DoubleType.'

In [54]:
# MinMaxScaler -  貌似输入列得是个matrix
trans = trans.na.fill({'amount':0}).withColumn('amount',col('amount').cast('double'))
scaler = MinMaxScaler(inputCol = 'amount', outputCol='amountScaler')
scalerModel =  scaler.fit(trans)
trans = scalerModel.transform(trans)
trans.show(5)

IllegalArgumentException: u'requirement failed: Column amount must be of type org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7 but was actually DoubleType.'

In [16]:
# 添加列，重命名一个列
trans.withColumn('try',trans.trans_id + 100).show(2)
trans.withColumnRenamed('amount','try2').show(2)

+----------+--------+----------+----------+-------+---------+---------+------+--------+
|   account|trans_id|trans_name|trans_date|prod_id|prod_name|trans_way|amount|     try|
+----------+--------+----------+----------+-------+---------+---------+------+--------+
|1003089018|  100541|    智能定投开通|  20161017| 020003|   国泰金龙行业|     网上银行|1000.0|100641.0|
|1001172983|  100003|    银行账户变更|  20161017|       |     null|     网上银行|   0.0|100103.0|
+----------+--------+----------+----------+-------+---------+---------+------+--------+
only showing top 2 rows

+----------+--------+----------+----------+-------+---------+---------+------+
|   account|trans_id|trans_name|trans_date|prod_id|prod_name|trans_way|  try2|
+----------+--------+----------+----------+-------+---------+---------+------+
|1003089018|  100541|    智能定投开通|  20161017| 020003|   国泰金龙行业|     网上银行|1000.0|
|1001172983|  100003|    银行账户变更|  20161017|       |     null|     网上银行|   0.0|
+----------+--------+----------+----------+-------+-

In [33]:
# trans.select(trans.trans_date).rdd.min()
# trans.agg(func.max('trans_date'),func.min('trans_date')).collect()
int((trans.agg({'trans_date':'max','trans_date':'min'}).collect())[0]['min(trans_date)'])

20161017

In [92]:
# trans.select(func.date_format('trans_date','MM-dd-yyy').alias('date')).show()
?trans.cube('account')

In [101]:
trans2 = trans.select(trans.account.alias('acc'),trans.trans_date.alias('dd'))
new_trans= trans.join(trans2,trans.account == trans2.acc, "inner")
sqlCtx.registerDataFrameAsTable(new_trans,'new_trans')

In [132]:
# 字符串转时间
times = spark.sql("select account, (from_unixtime(unix_timestamp(trans_date,'yyyyMMdd'))) date1,(from_unixtime(unix_timestamp(dd,'yyyyMMdd'))) date2 from new_trans")
sqlCtx.registerDataFrameAsTable(times,'times')

In [140]:
spark.sql("select account , (date1 - date2) dd from times where date1 > date2 order by dd desc").show()

+----------+----+
|   account|  dd|
+----------+----+
|1000493981|null|
|1000017846|null|
|1000017846|null|
|1000017846|null|
|1001864565|null|
|1001864565|null|
|1003068982|null|
|1003068982|null|
|1001864565|null|
|1001864565|null|
|1000000253|null|
|1000000253|null|
|1001088222|null|
|1001225424|null|
|1001920928|null|
|1000000253|null|
|1000000253|null|
|1001298654|null|
|1001298654|null|
|1000017846|null|
+----------+----+
only showing top 20 rows



In [96]:
trans.groupBy('account').pivot('trans_name',['trans_way']).count('prod_id').show()

TypeError: _api() takes exactly 1 argument (2 given)

In [92]:
tr = trans.rdd.map(lambda fields:fields.amount).map(lambda x:int(x))
# 返回了个defaultdict

tr.filter(lambda x: x > 10).countByValue()

defaultdict(int,
            {20: 2,
             30: 1,
             50: 6,
             60: 1,
             80: 7,
             100: 6712,
             120: 6,
             125: 4,
             150: 29,
             160: 1,
             200: 2677,
             250: 110,
             300: 4000,
             313: 1,
             314: 1,
             340: 8,
             348: 2,
             350: 18,
             360: 1,
             365: 1,
             398: 2,
             400: 6832,
             401: 4,
             403: 8,
             420: 1,
             450: 12,
             470: 3,
             499: 1,
             500: 7178,
             501: 1,
             502: 1,
             503: 8,
             505: 6,
             520: 7,
             550: 1,
             560: 1,
             600: 477,
             634: 1,
             690: 1,
             700: 61,
             750: 2,
             777: 1,
             800: 373,
             900: 27,
             1000: 2412,
             

In [88]:
# 排序， 还有orderby
cost.sort([u'产品代码',u'投资成本'],ascending = [True,True]).show()
cost.orderBy([u'产品代码',u'投资成本'],ascending = [True,True]).show()

+----------+------+--------+
|       客户号|  产品代码|    投资成本|
+----------+------+--------+
|1003281382|   100|  1500.0|
|1000744878|100016|     0.0|
|1000295034|100018| 15000.0|
|1001601251|100018|800000.0|
|1001244454|100020|  1500.0|
|1000004109|100021|     0.0|
|1002833265|100022|     0.0|
|1000977819|100022|     0.0|
|1002410662|100022|     0.0|
|1000706323|100022|     0.0|
|1001162745|100022|     0.0|
|1001292053|100022|     0.0|
|1002068170|100022| 10000.0|
|1002803375|100022|  1200.0|
|1001731895|100022|  1500.0|
|1000630433|100022| 15000.0|
|1001882209|100022|  2000.0|
|1001875108|100022|  2100.0|
|1002000342|100022|  2400.0|
|1000275783|100022| 28000.0|
+----------+------+--------+
only showing top 20 rows

+----------+------+--------+
|       客户号|  产品代码|    投资成本|
+----------+------+--------+
|1003281382|   100|  1500.0|
|1000744878|100016|     0.0|
|1000295034|100018| 15000.0|
|1001601251|100018|800000.0|
|1001244454|100020|  1500.0|
|1000004109|100021|     0.0|
|1002833265|10002

In [63]:
# 选择莫一列，重命名列
new_cost = cost.select(cost[u'产品代码'].alias('prod_id'), (cost[u'投资成本'] + 10).alias('age'))
new_cost.registerTempTable('new_cost')

In [114]:
?new_cost.alias

In [98]:

new_cost.groupBy('prod_id').agg(func.avg("age").alias('avg_age')).orderBy('avg_age',ascending = False).show()

+-------+------------------+
|prod_id|           avg_age|
+-------+------------------+
| 100018|          407510.0|
| 690012|          380010.0|
|    747|        356085.155|
|    177| 355343.3333333333|
|  40018|          347010.0|
|   1763|          332010.0|
|   2197|          275010.0|
|  20022|215849.91999999998|
| 240008|          207010.0|
|   1534|          200010.0|
| 270004|172605.33333333334|
| 110050|         169841.82|
|    222|157198.88888888888|
|    647|         156096.75|
|    221|154901.30434782608|
|   2946|153343.33333333334|
|    235|149535.38461538462|
| 110007|122937.27272727272|
|   1917|          120460.0|
|   1816|          120010.0|
+-------+------------------+
only showing top 20 rows



In [93]:
?func.bin()

In [52]:
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |       cc|       true|
|        |     cost|       true|
|        | customer|       true|
|        | new_cost|       true|
|        |  product|       true|
|        |     test|       true|
|        |    trans|       true|
+--------+---------+-----------+



In [49]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

IllegalArgumentException: u'requirement failed: key cannot be null'

In [54]:
# trans.createGlobalTempView('trans_new')
spark.sql("SELECT * FROM global_temp.trans_new").show()

+----------+------+------+--------+------+--------+----+------+
|      客户编号|  交易代码|  交易名称|    交易日期|  产品代码|    产品名称|交易渠道|  交易金额|
+----------+------+------+--------+------+--------+----+------+
|1003089018|100541|智能定投开通|20161017|020003|  国泰金龙行业|网上银行|1000.0|
|1001172983|100003|银行账户变更|20161017|      |    null|网上银行|   0.0|
|1001232639|100208|定期定额开通|20161017|100056|  富国低碳环保|网上银行| 100.0|
|1001232639|100208|定期定额开通|20161017|001616|  嘉实环保低碳|网上银行| 100.0|
|1003275159|100541|智能定投开通|20161017|000136|民生策略精选混合|网上银行| 400.0|
|1003276023|100208|定期定额开通|20161017|340007|  兴全社会责任|网上银行| 100.0|
|1003276005|100208|定期定额开通|20161017|340007|  兴全社会责任|网上银行| 100.0|
|1000068852|100541|智能定投开通|20161017|320011|   诺安中小盘|手机银行| 600.0|
|1002605841|100208|定期定额开通|20161017|020003|  国泰金龙行业|网上银行| 500.0|
|1003276582|100208|定期定额开通|20161017|000136|民生策略精选混合|网上银行|1000.0|
|1003267717|100208|定期定额开通|20161017|100056|  富国低碳环保|网上银行| 100.0|
|1003276954|100541|智能定投开通|20161017|000136|民生策略精选混合|网上银行| 400.0|
|1002978893|200208|定期定额处理|20161017|00045

In [78]:
from pyspark.sql import Row
age = new_cost.rdd.map(lambda p: "Name: " + str(p['age'])).collect()
prod = new_cost.rdd.map(lambda p: "Name: " + p.prod_id).collect()

In [82]:
age[0]

'Name: 4510.0'

In [50]:
sc.getAll()

[(u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.app.name', u'PySparkShell')]

In [14]:
lines = sc.parallelize(["hello word","hi"])
words = lines.flatMap(lambda line:line.split(" "))
print words.collect()
words = lines.map(lambda line:line.split(" "))
words.collect()

['hello', 'word', 'hi']


[['hello', 'word'], ['hi']]

In [18]:
?lines.aggregate()

In [24]:
# 分布式，comb是把各个分去再合并
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp)

(10, 4)

In [34]:
customer.take(5)

[Row(account=u'1000162168', cus_type=u'\u4e2a\u4eba', cus_group=u'\u666e\u901a\u5ba2\u6237', certification=u'\u8eab\u4efd\u8bc1', gender=u'\u5973', risk=u'\u4fdd\u5b88\u578b', birthday=u'19560919', manager=u'6000493'),
 Row(account=u'1000162203', cus_type=u'\u4e2a\u4eba', cus_group=u'\u666e\u901a\u5ba2\u6237', certification=u'\u8eab\u4efd\u8bc1', gender=u'\u7537', risk=u'\u4fdd\u5b88\u578b', birthday=u'19820630', manager=u' '),
 Row(account=u'1000190754', cus_type=u'\u4e2a\u4eba', cus_group=u'\u666e\u901a\u5ba2\u6237', certification=u'\u8eab\u4efd\u8bc1', gender=u'\u5973', risk=u'\u4fdd\u5b88\u578b', birthday=u'19551211', manager=u' '),
 Row(account=u'1000190759', cus_type=u'\u4e2a\u4eba', cus_group=u'\u666e\u901a\u5ba2\u6237', certification=u'\u8eab\u4efd\u8bc1', gender=u'\u7537', risk=u'\u4fdd\u5b88\u578b', birthday=u'19550529', manager=u' '),
 Row(account=u'1000190779', cus_type=u'\u4e2a\u4eba', cus_group=u'\u666e\u901a\u5ba2\u6237', certification=u'\u8eab\u4efd\u8bc1', gender=u'\u5

In [29]:
customer = customer.rdd.coalesce(3)

In [36]:
cost.show()

+----------+-------+-------+
|   account|prod_id| amount|
+----------+-------+-------+
|1000000132|    942| 4500.0|
|1000000132|   1180| 4500.0|
|1000000132|    823|47000.0|
|1000000132|   1150|24000.0|
|1000000140| 470008|    0.0|
|1000000161|  50026|  400.0|
|1000000161|   2644|    0.0|
|1000000161|   1743|10000.0|
|1000000161|   2560|    0.0|
|1000000161| 240020|  400.0|
|1000000161|     83| 4000.0|
|1000000161|   1125| 4000.0|
|1000000167| 320015| 6000.0|
|1000000167| 161655| 9000.0|
|1000000170| 161654|22000.0|
|1000000170| 270010|22000.0|
|1000000182| 470009|10000.0|
|1000000182|   1125| 2000.0|
|1000000188| 470009|46000.0|
|1000000188|  70022|    0.0|
+----------+-------+-------+
only showing top 20 rows



In [56]:
acc_prod = cost.rdd.map(lambda x:(x.account,x.prod_id))
acc_amount = cost.rdd.map(lambda x:(x.account,x.amount))
acc_prod_j = acc_prod.mapValues(lambda x: x + 'haha')
acc_amount_j = acc_amount.mapValues(lambda x: x + 'heihei')
a_p_j = acc_prod_j.join(acc_amount_j)

In [54]:
from datetime import datetime
?datetime.strptime()

In [73]:
aa = sqlContext.createDataFrame(a_p_j)

In [75]:
aa.columns

['_1', '_2']

In [5]:
cpd = customer.toPandas()
cpd.head()

,account,cus_type,cus_group,certification,gender,risk,birthday,manager
0,1000162168,个人,普通客户,身份证,女,保守型,19560919,6000493
1,1000162203,个人,普通客户,身份证,男,保守型,19820630,
2,1000190754,个人,普通客户,身份证,女,保守型,19551211,
3,1000190759,个人,普通客户,身份证,男,保守型,19550529,
4,1000190779,个人,普通客户,身份证,女,保守型,19831222,
